In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from tqdm import tnrange, tqdm_notebook, tqdm

import skorch
import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

import numpy as np
import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
from Lung_dataset import ILDDataset

In [ ]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

In [ ]:
NUM_TOTAL = 1968
NUM_TRAIN = 1783

#add path as absolute path for root dir

im_size = 128

lung_dataset_train = ILDDataset(csv_file=r'C:/Users/Akrofi/Desktop/CS 231/Project/train_labels.csv', 
                          root_dir=r'C:/Users/Akrofi/Desktop/CS 231/Project/train',mask=True, train=True, resize=im_size)#, transform=transform)

lung_dataset_validation = ILDDataset(csv_file=r'C:/Users/Akrofi/Desktop/CS 231/Project/train_labels.csv', 
                           root_dir=r'C:/Users/Akrofi/Desktop/CS 231/Project/validation',mask=True, train=False, resize=im_size)#, transform=transform)

#add path as absolute path for root dir
lung_dataset_test = ILDDataset(csv_file=r'C:/Users/Akrofi/Desktop/CS 231/Project/test_labels.csv', 
                          root_dir=r'C:/Users/Akrofi/Desktop/CS 231/Project/test', mask=True, train=False, resize=im_size)#, transform=transform)


weight0= 1/35
weight1= 1/18
weight2= 1/47

weights = []
for i in tnrange(1783):
    sample = lung_dataset_train[i]
    if sample[1] == 0:
        weights.append(weight0)
        
    elif sample[1] == 1:
        weights.append(weight1)
    else:
        weights.append(weight2)

sampler = sampler.WeightedRandomSampler(torch.cuda.DoubleTensor(weights), len(weights) )


In [ ]:
loader_train = DataLoader(lung_dataset_train, batch_size= 16,
                          sampler = sampler)

loader_val = DataLoader(lung_dataset_validation, batch_size=16,
                        sampler = torch.utils.data.sampler.SubsetRandomSampler(range(190)))

loader_test = DataLoader(lung_dataset_test, batch_size=16, sampler = torch.utils.data.sampler.SubsetRandomSampler(range(370)))

In [ ]:
#show datasample
sample = lung_dataset_test[374]
plt.imshow(sample[0], cmap='gray')
plt.show()
print("label: " + str(sample[1]))

In [ ]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)


In [ ]:
def get_model(freeze=False):
    """
    Used to fetch model for classification
    """

    num_classes = 3

    model = models.resnet50(pretrained=True)
    model.avgpool = nn.AvgPool2d(1, stride=1)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=1, bias=False)
    if(freeze==True):
        for param in model.parameters():
            param.requires_grad = False
    model.fc = nn.Linear(32768, num_classes)
    
    return model
    

In [ ]:
##### From CS 231N
def check_accuracy(loader, model, train=False, val = False):
    predictedStore = []
    solutionStore = []
    if loader.dataset.train and train == True:
        print('Checking accuracy on training set')
    elif val:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            x.unsqueeze_(1)
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            predictedStore.append(preds.view(-1,).cpu().numpy().tolist())
            solutionStore.append(y.data.view(-1,).cpu().numpy().tolist())
            
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples      
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        
    return acc,predictedStore,solutionStore
 

In [ ]:
##### From CS 231N
def train(model, optimizer, epochs=1, overfit=False):
    """
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    best_acc = 0;
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            
            # Implemet Weighted Loss
            weights = torch.tensor([1/35,1/18, 1/47], device= device, dtype=dtype)
            x.unsqueeze_(1)
            scores = model(x)
            Loss = nn.CrossEntropyLoss(weight = weights)
            loss = Loss(scores, y)


            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            
        print('Iteration %d, loss = %.4f' % (t, loss.item()))
        if(overfit):
            _,predictedStoreTrain,solutionStoreTrain = check_accuracy(loader_train, model, train=True)
        acc,predictedStoreVal,solutionStoreVal = check_accuracy(loader_val, model, val = True)
        if acc > best_acc:
            #Save Best Model
            best_acc = acc
            best_model = model
            predStoreTrain = predictedStoreTrain
            solStoreTrain = solutionStoreTrain
            predStoreVal = predictedStoreVal
            solStoreVal = solutionStoreVal
        print()
    return best_model,predStoreTrain,solStoreTrain,predStoreVal,solStoreVal

In [ ]:
# From CS 231N
model1 = get_model(freeze = True)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model1.parameters()), lr=0.0001, weight_decay= 1e-5)
best_model,predStoreTrain,solStoreTrain,predStoreVal,solStoreVal = train(model1, optimizer, epochs= 5, overfit=True)

In [ ]:
#Check Test set
acc,predStoreTest,solStoreTest = check_accuracy(loader_test, best_model)

In [ ]:
# From SciKit Documentation Page
import itertools
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    
    
##Actual Plots

y_pred_train = np.concatenate([np.array(i) for i in predStoreTrain])
y_train = np.concatenate([np.array(i) for i in solStoreTrain])

y_pred_val = np.concatenate([np.array(i) for i in predStoreVal]) 
y_val = np.concatenate([np.array(i) for i in solStoreVal])

y_pred_test = np.concatenate([np.array(i) for i in predStoreTest]) 
y_test = np.concatenate([np.array(i) for i in solStoreTest])

cnf_matrix1 = confusion_matrix(y_train, y_pred_train)
cnf_matrix2 = confusion_matrix( y_val, y_pred_val)
cnf_matrix3 = confusion_matrix(y_test, y_pred_test)



class_names = [0,1,2]
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix1, classes=class_names,
                      title='Normalized confusion matrix - Train')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix2, classes=class_names,
                      title='Normalized confusion matrix - Val')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix3, classes=class_names,
                      title='Normalized confusion matrix - Test')

plt.show()


In [ ]:
# From SciKit Documentation Page
    
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm, datasets
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

y_test = label_binarize(y_test, classes = [0,1,2])
y_pred_test = label_binarize(y_pred_test, classes = [0,1,2])
y_val = label_binarize(y_val, classes = [0,1,2])
y_pred_val = label_binarize(y_pred_val, classes = [0,1,2])

precision = dict()
recall = dict()
average_precision = dict()
n_classes = 3
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_val[:, i],
                                                        y_pred_val[:, i])
    average_precision[i] = average_precision_score(y_val[:, i], y_pred_val[:, i])

# Compute micro-average ROC curve and ROC area
precision["micro"], recall["micro"], _ = precision_recall_curve(y_val.ravel(),
    y_pred_val.ravel())
average_precision["micro"] = average_precision_score(y_val, y_pred_val,
                                                     average="micro")

# Plot Precision-Recall curve
plt.clf()
plt.plot(recall[0], precision[0], label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall example: AUC={0:0.2f}'.format(average_precision[0]))
plt.legend(loc="lower left")
plt.show()

# Plot Precision-Recall curve for each class
plt.clf()
plt.plot(recall["micro"], precision["micro"],
         label='micro-average Precision-recall curve (area = {0:0.2f})'
               ''.format(average_precision["micro"]))
for i in range(n_classes):
    plt.plot(recall[i], precision[i],
             label='Precision-recall curve of class {0} (area = {1:0.2f})'
                   ''.format(i, average_precision[i]))

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Extension of Precision-Recall curve to multi-class')
plt.legend(loc="lower right")
plt.show()

print(precision)
print(recall)